In [85]:
import pandas as pd
import numpy as np

In [86]:
demo_data = pd.read_csv(
    'data-prep-datasets/demoDetails.csv',
    index_col=0
    )

In [87]:
demo_data.head()
# demo_data.info()
# demo_data.shape
# demo_data.size

,customerID,gender,SeniorCitizen,Partner,Dependents
1,8260-NGFNY,Female,0.0,No,No
2,2359-QWQUL,Female,0.0,Yes,No
3,6598/RFFVI,Male,0.0,No,No
4,IXSTS-8780,Female,0.0,No,No
5,2674/MIAHT,Female,0.0,No,No


In [88]:
account_details = pd.read_csv(
    'data-prep-datasets/acDetails.txt',
    sep='\t',)
account_details.head()

,customerID,tenure,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
1,8260-NGFNY,One,Month-to-month,No,Mailed check,25.2,25.20
2,2359-QWQUL,39,One year,Yes,Credit card (automatic),104.7,4134.85
3,6598/RFFVI,2,One year,No,Credit card (automatic),19.3,28.30
4,IXSTS-8780,6,Month-to-month,Yes,Electronic check,90.1,521.30
5,2674/MIAHT,Four,Month-to-month,Yes,Mailed check,80.3,324.20


In [89]:
service_details = pd.read_csv(
    'data-prep-datasets/serviceDetails.csv',
    index_col=0,)
service_details.head()


,customerID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Churn
1,8260-NGFNY,No,No phone service,DSL,No,No,No,No,No,No,Yes
2,2359-QWQUL,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Yes
3,6598/RFFVI,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes
4,IXSTS-8780,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Yes
5,2674/MIAHT,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,No


# Data Wrangling
* merge account_details, demo_details and service_details.
before merging need to make the necessary checks to the data.

* Check for duplicate records and remove them.
* Whether the customer ID is common across all the files.



In [90]:
# Checking for duplicate records using numpy
len(np.unique(
    demo_data['customerID']
))
len(np.unique(
    account_details['customerID']
))
len(np.unique(
    service_details['customerID']
))

250

In [91]:
# checking for duplicate records using pandas
print(
    f"""
    {demo_data.duplicated().sum()}
    {account_details.duplicated().sum()}
    {service_details.duplicated().sum()}
    
    """
)


    1
    1
    1
    
    


In [92]:
demo_data.duplicated(
    subset=['customerID'],
    keep=False
)

1      False
2      False
3      False
4      False
5      False
       ...  
247    False
248    False
249    False
250    False
251     True
Length: 251, dtype: bool

In [93]:
# Check each row using subset and look for duplicates.
# If there are duplicates, then drop the row.
demo_data[demo_data.duplicated(
    subset=['customerID'],
    keep=False)]
account_details[account_details.duplicated(
    subset=['customerID'],
    keep=False)]
service_details[service_details.duplicated(
    subset=['customerID'],
    keep=False)]

,customerID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Churn
46,9067-SQTNS,Yes,No,No,Yes,Yes,No internet service,No internet service,Yes,No internet service,No
251,9067-SQTNS,Yes,No,No,Yes,Yes,No internet service,No internet service,Yes,No internet service,No


In [94]:
# removing the duplicates
#  First occurance of the duplicate will be removed.

demo_data = demo_data.drop_duplicates()
account_details = account_details.drop_duplicates()
service_details = service_details.drop_duplicates()

In [95]:
print(
    f"""
    {demo_data.shape}
{account_details.shape}
{service_details.shape}
    """
)



    (250, 5)
(250, 7)
(250, 11)
    


In [96]:
# Check if two dataframes are equal or not.
demo_data.customerID.equals(
    account_details.customerID)

True

In [97]:
account_details.customerID.equals(
    service_details.customerID)

True

In [98]:
demo_data.customerID.equals(
    service_details.customerID)


True

Joining two data frames based on a condition ie (customerId)

In [99]:
churn = pd.merge(
    demo_data,
    account_details,
    on='customerID',
)
churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,8260-NGFNY,Female,0.0,No,No,One,Month-to-month,No,Mailed check,25.2,25.20
1,2359-QWQUL,Female,0.0,Yes,No,39,One year,Yes,Credit card (automatic),104.7,4134.85
2,6598/RFFVI,Male,0.0,No,No,2,One year,No,Credit card (automatic),19.3,28.30
3,IXSTS-8780,Female,0.0,No,No,6,Month-to-month,Yes,Electronic check,90.1,521.30
4,2674/MIAHT,Female,0.0,No,No,Four,Month-to-month,Yes,Mailed check,80.3,324.20


In [100]:
# merge churn with service details
churn = pd.merge(
    churn,
    service_details,
    on='customerID',
)
churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,...,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Churn
0,8260-NGFNY,Female,0.0,No,No,One,Month-to-month,No,Mailed check,25.2,...,No,No phone service,DSL,No,No,No,No,No,No,Yes
1,2359-QWQUL,Female,0.0,Yes,No,39,One year,Yes,Credit card (automatic),104.7,...,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Yes
2,6598/RFFVI,Male,0.0,No,No,2,One year,No,Credit card (automatic),19.3,...,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes
3,IXSTS-8780,Female,0.0,No,No,6,Month-to-month,Yes,Electronic check,90.1,...,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Yes
4,2674/MIAHT,Female,0.0,No,No,Four,Month-to-month,Yes,Mailed check,80.3,...,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,No


In [101]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        250 non-null    object 
 1   gender            250 non-null    object 
 2   SeniorCitizen     245 non-null    float64
 3   Partner           250 non-null    object 
 4   Dependents        250 non-null    object 
 5   tenure            250 non-null    object 
 6   Contract          250 non-null    object 
 7   PaperlessBilling  250 non-null    object 
 8   PaymentMethod     250 non-null    object 
 9   MonthlyCharges    240 non-null    float64
 10  TotalCharges      235 non-null    float64
 11  PhoneService      250 non-null    object 
 12  MultipleLines     250 non-null    object 
 13  InternetService   250 non-null    object 
 14  OnlineSecurity    250 non-null    object 
 15  OnlineBackup      250 non-null    object 
 16  DeviceProtection  250 non-null    object 
 1

## Understanding the data.

In [102]:
churn


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,...,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Churn
0,8260-NGFNY,Female,0.0,No,No,One,Month-to-month,No,Mailed check,25.20,...,No,No phone service,DSL,No,No,No,No,No,No,Yes
1,2359-QWQUL,Female,0.0,Yes,No,39,One year,Yes,Credit card (automatic),104.70,...,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Yes
2,6598/RFFVI,Male,0.0,No,No,2,One year,No,Credit card (automatic),19.30,...,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes
3,IXSTS-8780,Female,0.0,No,No,6,Month-to-month,Yes,Electronic check,90.10,...,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Yes
4,2674/MIAHT,Female,0.0,No,No,Four,Month-to-month,Yes,Mailed check,80.30,...,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,8111-RKSPX,Male,0.0,No,No,Four,Month-to-month,Yes,Bank transfer (automatic),72.20,...,Yes,Yes,Fiber optic,No,No,No,No,No,No,Yes
246,6922-NCEDI,Male,0.0,No,Yes,56,One year,Yes,Mailed check,21.20,...,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No
247,5192-EBGOV,Female,1.0,No,No,One,Month-to-month,Yes,Electronic check,NaN,...,Yes,Yes,Fiber optic,No,No,No,No,Yes,No,Yes
248,8495-PRWFH,Female,1.0,No,No,42,Month-to-month,No,Electronic check,55.65,...,Yes,Yes,DSL,No,No,Yes,No,No,No,No


In [103]:
churn1 = churn.copy()
# churn1.head()

In [104]:
# Checking the information of the dataframe
churn1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        250 non-null    object 
 1   gender            250 non-null    object 
 2   SeniorCitizen     245 non-null    float64
 3   Partner           250 non-null    object 
 4   Dependents        250 non-null    object 
 5   tenure            250 non-null    object 
 6   Contract          250 non-null    object 
 7   PaperlessBilling  250 non-null    object 
 8   PaymentMethod     250 non-null    object 
 9   MonthlyCharges    240 non-null    float64
 10  TotalCharges      235 non-null    float64
 11  PhoneService      250 non-null    object 
 12  MultipleLines     250 non-null    object 
 13  InternetService   250 non-null    object 
 14  OnlineSecurity    250 non-null    object 
 15  OnlineBackup      250 non-null    object 
 16  DeviceProtection  250 non-null    object 
 1

In [105]:
# check for missing values in the dataframe
# churn1.isnull().sum()

np.unique(
    churn1['tenure'],
    # return_counts=True
)


array(['10', '1008', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '21', '22', '25', '27', '28', '29', '3', '30', '31', '32',
       '34', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '6', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '7', '70', '71', '72', '8', '9', '964', '969',
       '971', '992', '994', 'Four', 'One'], dtype=object)

In [106]:
# select the cols according to the dtypes.
categorical_data = churn1.select_dtypes(
    include=[
        # 'float64', 
        # 'int64',
        'object',
        ]
)
categorical_data.columns

Index(['customerID', 'gender', 'Partner', 'Dependents', 'tenure', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Churn'],
      dtype='object')

In [107]:
categorical_data['gender'].value_counts()

Male      126
Female    124
Name: gender, dtype: int64

In [121]:
categorical_data.value_counts()
# categorical_data['customerID'].value_counts()
categorical_data['tenure'].value_counts()

# categorical_data = categorical_data.drop(
#     ['customerId', 'tenure'],
#     axis=1
# )

One     20
72      15
2        9
Four     9
9        8
        ..
30       1
54       1
47       1
994      1
971      1
Name: tenure, Length: 71, dtype: int64

In [117]:
freq_table = categorical_data.apply(
    lambda x: x.value_counts()).T.stack()
print(freq_table)

customerID       0115-TFERT               1.0
                 0187-WZNAB               1.0
                 0193-ESZXP               1.0
                 0195-IESCP               1.0
                 0203-HHYIJ               1.0
                                        ...  
StreamingMovies  No                     104.0
                 No internet service     61.0
                 Yes                     85.0
Churn            No                     172.0
                 Yes                     78.0
Length: 365, dtype: float64


* cleaning data.

In [123]:
# convert tenure to integer from object
# find the string values in the column
churn1.tenure.replace('One', "4" , inplace=True)
churn1.tenure.replace('Four', "4" , inplace=True)
churn1.tenure = churn1.tenure.astype(int)
# churn1.tenure.unique()


In [130]:
# clearning dependants col.
churn1['Dependents'].unique()
churn1['Dependents'].replace(
    '1@#',
    'No',
    inplace=True
)
churn1['Dependents'].unique()

array(['No', 'Yes'], dtype=object)

In [135]:
pd.crosstab(
    churn1['Dependents'],
    columns="count"
)


col_0,count
Dependents,
No,177
Yes,73


In [124]:

np.unique(
    churn1['tenure'],
    return_counts=True
    )


(array([   2,    3,    4,    5,    6,    7,    8,    9,   10,   11,   12,
          13,   14,   15,   16,   17,   18,   19,   21,   22,   25,   27,
          28,   29,   30,   31,   32,   34,   36,   37,   38,   39,   40,
          41,   42,   43,   44,   45,   46,   47,   48,   49,   50,   51,
          52,   53,   54,   55,   56,   57,   58,   59,   61,   62,   63,
          64,   65,   66,   67,   68,   69,   70,   71,   72,  964,  969,
         971,  992,  994, 1008]),
 array([ 9,  6, 29,  6,  3,  5,  5,  8,  5,  3,  1,  3,  2,  4,  5,  1,  2,
         2,  3,  5,  5,  5,  5,  2,  1,  2,  1,  1,  1,  3,  1,  4,  1,  1,
         3,  3,  4,  3,  2,  1,  3,  5,  2,  2,  3,  1,  1,  4,  2,  2,  5,
         1,  3,  5,  4,  7,  4,  3,  1,  3,  3,  5,  4, 15,  1,  1,  1,  1,
         1,  1]))

In [132]:
# find the count of all the nan values.
churn1['SeniorCitizen'].unique()
np.unique(
    churn1['SeniorCitizen'],
)
# churn1.info()

array([ 0.,  1., nan])